In [ ]:
pip install mysql-connector-python

In [51]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [52]:
pip install mysql-connector

     |████████████████████████████████| 11.9 MB 2.0 MB/s eta 0:00:01
  Created wheel for mysql-connector: filename=mysql_connector-2.2.9-cp39-cp39-macosx_10_9_x86_64.whl size=247953 sha256=e8a278d8b29107d2e116ecff0f82610d62c098531a8aefd55c28fc1086f0539f
  Stored in directory: /Users/isaacflath/Library/Caches/pip/wheels/7b/14/39/5aad423666e827dfe9a1fbcd111ac17171e7c9865d570780ce
Successfully built mysql-connector
Note: you may need to restart the kernel to use updated packages.


In [58]:
# pip install mysqlclient

In [59]:
# from sqlalchemy import create_engine

# db_connect_string=f"mysql+mysqldb://root:{config['password']}@{config['host']}:3306/political_influence"
# ssl_args = {'ssl': {'cert':config['ssl_cert'], 'key':config['ssl_key'], 'ca':config['ssl_ca']}}
# engine = create_engine(db_connect_string, connect_args=ssl_args)

# print(engine.table_names())

In [102]:
import os

In [113]:
import mysql.connector
from mysql.connector.constants import ClientFlag

config = {
    'user': 'root',
    'password': '0eD0fC65pknAb2ud',
    'host': '35.221.43.230',
    'client_flags': [ClientFlag.SSL],
    'ssl_ca': '/Users/isaacflath/ssh/server-ca.pem',
    'ssl_cert': '/Users/isaacflath/ssh/client-cert.pem',
    'ssl_key': '/Users/isaacflath/ssh/client-key.pem',
    'database':'political_influence'
}

In [114]:
cnxn = mysql.connector.connect(**config)

In [115]:
cursor = cnxn.cursor()  # initialize connection cursor

In [134]:
# cursor.execute('describe (select * from officers limit 5)')
# cols = cursor.fetchall()
cursor.execute('select * from officers limit 5')
out = cursor.fetchall()
display(pd.DataFrame(out,columns=[i[0] for i in cursor.description]))


,record_type,form_id_number,director_id,org_name,ein,entity_name,entity_title,entity_address_1,entity_address_2,entity_address_city,entity_address_st,entity_address_zip_code,entity_address_zip_code_ext
0,D,305,42980,WE THE PEOPLE TRUST INTERNATIONAL,11111111,RANDY POMBO,TRUSTEE,28375 S. CHRISMAN RD.,None,TRACY,CA,95304,None
1,D,305,42981,WE THE PEOPLE TRUST INTERNATIONAL,11111111,CLIVE WILSHIRE,TRUSTEE,38 CLEVES WAY,None,HAMPTON,CA,94001,None
2,D,305,42982,WE THE PEOPLE TRUST INTERNATIONAL,11111111,GEORGE GLADIUS,TRUSTEE,21 NO. SCHOOL ST.,None,LODI,CA,95240,None
3,D,305,42983,WE THE PEOPLE TRUST INTERNATIONAL,11111111,ABDULLAYA TENYI,TRUSTEE,RUE AEROTRANSIT MATOTOE,None,"CONAKRY, GUINEA",CA,95001,None
4,D,305,42984,WE THE PEOPLE TRUST INTERNATIONAL,11111111,SPENCER D. LEWIS,TRUSTEE,4935 N. JACK TONE RD.,None,STOCKTON,CA,95215,None


In [127]:
num_fields = len(cursor.description)
field_names = [i[0] for i in cursor.description]

In [129]:
field_names

['record_type',
 'form_id_number',
 'director_id',
 'org_name',
 'ein',
 'entity_name',
 'entity_title',
 'entity_address_1',
 'entity_address_2',
 'entity_address_city',
 'entity_address_st',
 'entity_address_zip_code',
 'entity_address_zip_code_ext']

In [ ]:
cursor.execute('CREATE DATABASE political_influence')  # create a new 'testdb' database

In [20]:
cnxn.close()  # close connection because we will be reconnecting to testdb

In [21]:
config['database'] = 'irs'  # add new database to config dict
cnxn = mysql.connector.connect(**config)
cursor = cnxn.cursor()

In [118]:
import pandas as pd
df = pd.read_pickle('outputs/directorssample.pickle')
df.head(2)

,record_type,form_id_number,director_id,org_name,ein,entity_name,entity_title,entity_address_1,entity_address_2,entity_address_city,entity_address_st,entity_address_zip_code,entity_address_zip_code_ext
787,D,305.0,42980.0,WE THE PEOPLE TRUST INTERNATIONAL,011111111,RANDY POMBO,TRUSTEE,28375 S. CHRISMAN RD.,None,TRACY,CA,95304,None
788,D,305.0,42981.0,WE THE PEOPLE TRUST INTERNATIONAL,011111111,CLIVE WILSHIRE,TRUSTEE,38 CLEVES WAY,None,HAMPTON,CA,94001,None


In [119]:
cursor.execute('''
                CREATE TABLE officers (
                    record_type                     VARCHAR(4),
                    form_id_number                  int,
                    director_id                     int,
                    org_name                        VARCHAR(256),
                    ein                             int,
                    entity_name                     VARCHAR(256),
                    entity_title                    VARCHAR(256),
                    entity_address_1                VARCHAR(256),
                    entity_address_2                VARCHAR(256),
                    entity_address_city             VARCHAR(256),
                    entity_address_st               VARCHAR(16),
                    entity_address_zip_code         VARCHAR(16),
                    entity_address_zip_code_ext     VARCHAR(16)
                )
                '''
              )

cnxn.commit() 

In [120]:
for col in df.columns: df[col] = df[col].astype(str)

In [121]:
# first we setup our query
query = '''
INSERT INTO officers (record_type, form_id_number, director_id, org_name, ein, entity_name, entity_title, entity_address_1, entity_address_2,
                      entity_address_city, entity_address_st, entity_address_zip_code, entity_address_zip_code_ext)
         VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''
# then we execute with every row in our dataframe
cursor.execute('truncate table officers')
cursor.executemany(query, list(df.to_records(index=False)))
cnxn.commit()  # and commit changes

In [108]:
# cursor.execute('describe (select * from officers limit 5)')
# cols = cursor.fetchall()
cursor.execute('select * from officers limit 5')
out = cursor.fetchall()
pd.DataFrame(out,columns=[o[0] for o in cols])

OperationalError: 2055: Lost connection to MySQL server at '35.221.43.230:3306', system error: 1 [SSL: BAD_LENGTH] bad length (_ssl.c:2483)